In [ ]:
import os
import pickle
import random
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import datetime
from joblib import dump

print("All imports successful!")

# rcv1 = fetch_rcv1()

ModuleNotFoundError: No module named 'sklearn'

In [2]:

# pickle.dump(rcv1.data, open('../data/data.pickle', 'wb'))
# pickle.dump(rcv1.target, open('../data/target.pickle', 'wb'))

In [3]:
# Check if the file exists within the folder
if os.path.exists('../data/'): 
    X = pickle.load(open('../data/data.pickle', 'rb'))
    y = pickle.load(open('../data/target.pickle', 'rb'))
    y = y.toarray()[:, random.randint(0, 3)]

else:
    rcv1 = fetch_rcv1()
    pickle.dump(open('../data/data.pickle', 'w'), rcv1.data)
    pickle.dump(open('../data/target.pickle', 'w'), rcv1.target)  
    X = rcv1.data  
    y = rcv1.target.toarry()[:, random.randint(0, 3)]

    

In [4]:

mlflow.set_tracking_uri("./mlruns")
dataset_name = "Reuters Corpus Volume"
current_time = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
experiment_name = f"{dataset_name}_{current_time}"    
experiment_id = mlflow.create_experiment(f"{experiment_name}")

with mlflow.start_run(experiment_id=experiment_id,
                      run_name= f"{dataset_name}"):
    
    params = {
                "dataset_name": dataset_name,
                "number of datapoint": X.shape[0],
                "number of dimensions": X.shape[1]}
    
    mlflow.log_params(params)
    
    train_X, test_X, train_y, test_y = sklearn.model_selection.train_test_split(X, y, 
                                                                                test_size = 0.9,
                                                                                shuffle = True)
    
    
    dt = DecisionTreeClassifier()
    dt.fit(train_X, train_y)
    
    y_predict = dt.predict(test_X)
    mlflow.log_metrics({'Accuracy': accuracy_score(test_y, y_predict),
                        'F1 Score': f1_score(test_y, y_predict)})
    
    if os.path.exists('../model/'): 
        dump(dt, f'../model/{experiment_id}.jopblib')
                       